# Test pour prédire le type de cancer (microarray data)
Flückiger Nathan, Vincent Guidoux et Joel Kaufmann

## Préparation des données

### Importation des bibliothèques

In [1]:
import os
import numpy as np
import pylab as plt
import pandas as pd

from Bio import Geo


%matplotlib inline

### Importation des données

In [2]:
def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

In [3]:
records = load_geo('GDS4296.soft')

# description of each condition (cell type)
id_cond = {}  

cancer_set = set()

# loop over records
for r in records:    
    
    rea = r.entity_attributes
    
    #if rea.has_key('subset_sample_id'): 
    if 'subset_sample_id' in rea:
        
        # find conditions associated with each cell type
        if rea['subset_type']=='disease state':
            sample_list = rea['subset_sample_id'].split(',')
            desc = rea['subset_description']
            cancer_set.add(desc)
            for sample in sample_list:
                id_cond[sample] = desc

    #if rea.has_key('dataset_table_begin'): # it's the gene expression data
    if 'dataset_table_begin' in rea:
        data = r.table_rows

# converts data to an array (more practical to handle)
data = np.array(data) 

**Combien y'a-t-il de cellules cancereuses en tout ?**

In [4]:
len(id_cond)

174

**Comment sont-elles réparties ?**

In [5]:
disease_repartition = {}

for disease in id_cond.values():
    if disease in disease_repartition.keys():
        disease_repartition[disease] = disease_repartition[disease] + 1
    else:
        disease_repartition[disease] = 1
        
disease_repartition

{'Adenocarcinoma': 60,
 'Acute lymphoblastic leukemia': 6,
 'Astrocytoma': 3,
 'Chronic myelogenous leukemia': 3,
 'Carcinoma': 12,
 'Carcinosarcoma': 3,
 'Clear cell carcinoma': 2,
 'Cystoadenocarcinoma': 3,
 'Ductal carcinoma': 3,
 'Glial cell neoplasm': 3,
 'Glioblastoma': 12,
 'Large cell carcinoma': 6,
 'Lymphoma': 3,
 'Malignant amelanotic melanoma': 3,
 'Malignant melanotic melanoma': 11,
 'Melanotic melanoma': 9,
 'Myeloma': 3,
 'Papillary infiltrating ductal carcinoma': 3,
 'Pro myelocytic leukemia': 3,
 'Renal spindle cell carcinoma': 3,
 'Renal cell carcinoma': 9,
 'Small cell bronchioalveolar carcinoma': 3,
 'Squamous cell carcinoma': 2,
 'Hypernephroma': 3,
 'Infiltrating ductal carcinoma': 3}

Aux vues de cette répartition, il nous faut regrouper ces différentes type de cellules, car certains, comme `Squamous cell carcinoma` n'ont que deux cellules, et on ne peut pas se permettre d'en mettre une dans le train-set et une dans le test-set, ce n'est pas assez. Nous allons nous renseigner comment regrouper ces données.

In [6]:
def category_repartition(category):
    """Vous nous aider à voir la répartition des différents échantillons dans les catégories que nous allons définir"""
    total = 0
    for disease in category:
        total += disease_repartition[disease]
        
    return total

**5 catégories ont été choisies :**

Adenocarcinoma :

In [7]:
category_1 = ['Adenocarcinoma']

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_1)))

Il y a 60 échantillons dans cette catégorie


Other Carcinoma :

In [8]:
category_2 = [
                'Carcinoma',
                'Carcinosarcoma',
                'Clear cell carcinoma',
                'Cystoadenocarcinoma',
                'Ductal carcinoma',
                'Large cell carcinoma',
                'Papillary infiltrating ductal carcinoma',
                'Renal spindle cell carcinoma',
                'Renal cell carcinoma',
                'Small cell bronchioalveolar carcinoma',
                'Squamous cell carcinoma',
                'Hypernephroma',
                'Infiltrating ductal carcinoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_2)))

Il y a 55 échantillons dans cette catégorie


Leucémie : 

In [9]:
category_3 = [
                'Acute lymphoblastic leukemia',
                'Chronic myelogenous leukemia',
                'Lymphoma',
                'Myeloma',
                'Pro myelocytic leukemia'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_3)))

Il y a 18 échantillons dans cette catégorie


Mélanome :

In [10]:
category_4 = [
                'Malignant amelanotic melanoma',
                'Malignant melanotic melanoma',
                'Melanotic melanoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_4)))

Il y a 23 échantillons dans cette catégorie


Cerveau :

In [11]:
category_5 = [
                'Astrocytoma',
                'Glial cell neoplasm',
                'Glioblastoma'
            ]

print("Il y a {} échantillons dans cette catégorie".format(category_repartition(category_5)))

Il y a 18 échantillons dans cette catégorie


**Vérifier si les écart-type et moyennes de ces catégories sont logiques, pour confirmer si on a bien choisi**

Pour ça, il faut qu'on rajoute une ligne `category` à `data` et lui attribuer `1,2,3,4 ou 5` ou trouver un autre moyen de les identifer pour cela on peut s'aider de `id_cond` qui map les `ID_REF` de style : `"GSM803615", "GSM803674", "GSM803733"` avec les différents cancers : `"Small cell bronchioalveolar carcinoma", "Squamous cell carcinoma"`.

Ensuite, il faut séparer les données de test et d'entraînement :

![coucou](./images/test_train_diagram.png)

Ce diagramme se trouve ici si on veut le modifier : [draw.io](https://drive.google.com/file/d/1mcjHukPsXNBmaIeI8rkmtCZdjN7z8OEm/view?usp=sharing)

Pour cette étape, on peut s'inspirer du code du labo 3-4 de BBC : 

In [12]:
# Labo 3-4 BBC :
# divide the set into two sets: a train set and a test set for future validations
# idx_test = [5,11,19,28,54,61]  # Soit on prend un pourcentage soit on en choisit genre 3 par catégories
# idx_train = list(range(0,len(id_cond)+2)) 
# [idx_train.remove(i) for i in idx_test] # on enlève les données de test

# data_test = data[:,idx_test]
# data = data[:,idx_train]

# print(np.shape(data))
# print(np.shape(data_test))

Et Maintenant il faut voir pour ces écart-type

un endroit par là, il y a les fonctions à Guidoux qui diminue le nombre de variable de 50'000 a plus petit, pourquoi on utilise pas les méthodes du labo 3-4 ? Parce que c'était un problème à deux classes, et là on est multi classe ?


Les guidoux méthodes sont dans ce [google colab notebook](https://colab.research.google.com/drive/1rWg2k7-RtgJfzjacYNNY33m00WGDRxRq) à 
+ Méthode 1 - Unvariant Selection
+ Methode 4 - Recursive features elimination

Mais au premier abord, il faudrait échanger les lignes et les colonnes, et mettre les catégories dans un autre tableau

Même s'il manque les méthodes à Guidoux, ce serait bien de mettre les méthodes à Joël pour la classification, au moins une des 3 : 

+ Decision tree classifier
+ Support vector machine classifier
+ k-nearest neighbours classifier
